In [1]:
# Core 
import datetime
import os
import glob
# from IPython.display import HTML, Image, display
import tempfile
import shutil
import sys
import pickle
import warnings

# Analysis 
import xarray as xr
import numpy as np
import pandas as pd

# Debugging 
import pdb, traceback
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint import animate
# from tint.visualization import embed_mp4_as_gif


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [49]:
def CPOL_files_from_datetime_dataframe(datetimes):
    filenames = []
    for i in range(len(datetimes)):
        year = str(datetimes.iloc[i])[0:4]
        month = str(datetimes.iloc[i])[5:7]
        day = str(datetimes.iloc[i])[8:10]
        hour = str(datetimes.iloc[i])[11:13]
        minute = str(datetimes.iloc[i])[14:16]
        filename = (base + '{0}/{0}{1}{2}/'.format(year, month, day) 
                    + 'CPOL_{0}{1}{2}'.format(year, month, day)
                    + '_{}{}_GRIDS_1000m.nc'.format(hour, minute))
        if os.path.isfile(filename):
            filenames.append(filename)
    
    return sorted(filenames), datetimes.iloc[0], datetimes.iloc[-1]

def CPOL_files_from_datetime_list(datetimes):
    filenames = []
    for i in range(len(datetimes)):
        year = str(datetimes[i])[0:4]
        month = str(datetimes[i])[5:7]
        day = str(datetimes[i])[8:10]
        hour = str(datetimes[i])[11:13]
        minute = str(datetimes[i])[14:16]
        filename = (base + '{0}/{0}{1}{2}/'.format(year, month, day) 
                    + 'CPOL_{0}{1}{2}'.format(year, month, day)
                    + '_{}{}_GRIDS_1000m.nc'.format(hour, minute))
        if os.path.isfile(filename):
            filenames.append(filename)
    
    return sorted(filenames), datetimes[0], datetimes[-1]
    
def CPOL_files_from_TINT_obj(tracks_obj, uid):
    base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'
    
    datetimes = tracks_obj.system_tracks.xs(uid, level='uid')
    datetimes = datetimes.reset_index(level='time')['time']
    [files, start_date, end_date] = CPOL_files_from_datetime_list(datetimes)
    
    return files, start_date, end_date

In [50]:
filenames = CPOL_files_from_datetime_list(np.arange(np.datetime64('2016-12-01'), 
                                          np.datetime64('2017-03-01'), 
                                          np.timedelta64(10, 'm')))[0]
filenames

['/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0000_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0010_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0020_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0030_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0040_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0050_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0100_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0110_GRIDS_1000m.nc',
 '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161201/CPOL_20161201_0120_

In [ ]:
def test():
    # Store filenames for an example day of data
    base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

    filenames = CPOL_files_from_datetime_list(np.arange(np.datetime64('2016-12-01'), 
                                              np.datetime64('2017-03-01'), 
                                              np.timedelta64(10, 'm')))[0]
    # Generate grid generator 
    # Note generators produce iterators
    # These are alternative to using lists and looping
    grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
             for fn in filenames)

    # Define settings for tracking
    settings = {
        'MIN_SIZE' : [15, 15, 15, 15],
        'FIELD_THRESH' : [45, 40, 35, 30],
        'ISO_THRESH' : [4, 4, 4, 4],
        'GS_ALT' : 3000,
        'LEVELS' : np.array(
            [[0, 2000], 
             [2000, 4000],
             [4000, 6000],
             [6000, 8000]]
        ),
        'TRACK_INTERVAL' : 1,
    }

    # Calculate high and low level tracks
    tracks_obj  = tint.Cell_tracks()

    for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS', 
                      'TRACK_INTERVAL', 'ISO_THRESH'
                     ]:
        tracks_obj.params[parameter] = settings[parameter]

    # Calculate tracks
    tracks_obj.get_tracks(grids)

    # Create directory for figures
    dt=str(datetime.datetime.now())[0:-7]
    dt=dt.replace(" ", "_")
    dt=dt.replace(":", "_")
    dt=dt.replace("-", "")  

    out_file_name = '/g/data/w40/esh563/CPOL_analysis/TINT_tracks/tracks_obj_{}.pkl'.format(dt)

    with open(out_file_name, 'wb') as f:
        pickle.dump(tracks_obj, f)
    
if __name__ == '__main__':
    try:
        test()
    except:
        extype, value, tb = sys.exc_info()
        traceback.print_exc()
        pdb.post_mortem(tb)

In [2]:
with open('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/tracks_obj_20190711_19_00_12.pkl', 'rb') as f:
    tracks_obj = pickle.load(f)

In [6]:
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

# base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

# # Note we only need grid generators for the grids that actually contain
# # the system!
# filenames = sorted(glob.glob(base + '2017/20171221/*_2[1-3]??_*.nc')
#                    + glob.glob(base + '2017/20161222/*_00??_*.nc'))

filenames, start_time, end_time = CPOL_files_from_TINT_object(tracks_obj, '30')

# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity')
         for fn in filenames)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")    

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_anim'

animate(tracks_obj, grids, out_path,
        start_datetime = start_time,
        end_datetime = end_time,
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True, keep_frames=False, dpi=100)

Animating from 2016-12-21 16:10:02 to 2016-12-21 19:30:06.
Plotting scan at 2016-12-21T19:30:06.000.
Creating GIF - may take a few minutes.


In [27]:
# Let's filter by tilt direction and magnitude
forward_tilt = ((-45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 45))
backward_tilt = ((-45 >= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                | (tracks_obj.system_tracks['sys_rel_tilt_dir'] >= 45))
left_tilt = ((45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 135))
right_tilt = ((-135 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= -45))
sig_tilt_mag = (tracks_obj.system_tracks['tilt_mag'] >= 2)

In [10]:
# Let's find the cells that were tracked for the most frames
tracks_obj.system_tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]

uid
9     23
41    21
62    17
26    13
65    12
24    11
20     9
68     9
23     8
69     8
dtype: int64

In [11]:
tracks_obj.system_tracks.xs('9', level='uid')

,,grid_x,grid_y,lon,lat,u,v,n_cores,semi_major,semi_minor,eccentricity,orientation,x_vert_disp,y_vert_disp,tilt_mag,vel_dir,tilt_dir,sys_rel_tilt_dir
scan,time,,,,,,,,,,,,,,,,,
35,2016-12-21 05:50:02,13.257,109.057,130.5206,-11.8977,NaN,NaN,1,7.635641,3.374091,0.897071,-84.936322,0.651,-0.754,0.996151,NaN,-49.192831,NaN
36,2016-12-21 06:00:02,24.066,100.918,130.6216,-11.9698,18.015000,-13.565000,1,13.816279,5.306301,0.923308,89.560581,1.087,-2.118,2.380650,-36.979148,-62.832198,-25.853049
37,2016-12-21 06:10:02,30.532,87.567,130.6858,-12.0868,10.776667,-22.251667,2,33.420501,6.043217,0.983516,-79.970847,0.188,-2.391,2.398380,-64.158708,-85.504186,-21.345478
38,2016-12-21 06:20:02,35.825,76.881,130.7316,-12.1858,8.821667,-17.810000,1,28.375726,4.524690,0.987205,-69.128725,1.360,-1.462,1.996758,-63.649820,-47.070031,16.579790
39,2016-12-21 06:30:06,41.167,71.220,130.7776,-12.2398,8.903333,-9.435000,1,33.858192,7.908200,0.972340,-81.993378,0.828,-0.532,0.984179,-46.660659,-32.721235,13.939425
40,2016-12-21 06:40:01,47.658,69.322,130.8420,-12.2578,10.818333,-3.163333,1,23.628923,7.532424,0.947829,-76.114667,2.112,-3.700,4.260346,-16.299177,-60.281808,-43.982631
41,2016-12-21 06:50:01,53.351,67.471,130.8880,-12.2758,9.488333,-3.085000,2,38.428041,6.264804,0.986622,-83.323451,1.238,-1.322,1.811168,-18.011216,-46.879343,-28.868128
42,2016-12-21 07:00:02,58.490,65.117,130.9340,-12.2938,8.565000,-3.923333,1,47.488276,6.398962,0.990880,-82.827755,0.971,-3.267,3.408244,-24.610895,-73.447284,-48.836389
43,2016-12-21 07:10:01,64.683,62.654,130.9984,-12.3118,10.321667,-4.105000,1,44.846269,7.996561,0.983974,-82.732307,1.128,-3.318,3.504498,-21.688066,-71.223824,-49.535758


In [38]:
uid = '1256'

filenames, start_time, end_time = file_names_from_object(tracks_obj, uid)

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_lagrangian'
animate(tracks_obj, grids, out_path, style='lagrangian', uid=uid, alt=6000,
        keep_frames=False, dpi=100)

Animating 15 frames
Plotting frame at 2017-02-05 21:30:01.000
Creating GIF - may take a few minutes.
